In [1]:
# This file aims to debug the newly created solvent diffusion model with EC drag (add one submodel variable for EC concentration)

In [2]:
import pybamm as pybamm;import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;import imageio
from scipy.io import savemat,loadmat;from pybamm import constants,exp;import matplotlib as mpl; fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
for k in range(0,1):
    mpl.rcParams["axes.labelsize"] = fs
    mpl.rcParams["axes.titlesize"] = fs
    mpl.rcParams["xtick.labelsize"] =  fs
    mpl.rcParams["ytick.labelsize"] =  fs
    mpl.rcParams["legend.fontsize"] =  fs
    mpl.rcParams['font.sans-serif'] = ['Times New Roman']
    mpl.rcParams['font.serif'] = ['Times New Roman']
    mpl.rcParams['axes.unicode_minus'] = False

In [3]:
# Global setting Niall's result:
#Total_Cycles = 1170; Cycle_bt_RPT = 78; Update_Cycles = 78;  Small_Loop =  int(Cycle_bt_RPT/Update_Cycles);   SaveTimes = int(Total_Cycles/Cycle_bt_RPT);   RPT_Cycles = 1; 

# Special cases for Non-dry-out model:
#Total_Cycles_NonDry = 1170-78;   SaveTimes_NonDry = int(Total_Cycles_NonDry/Cycle_bt_RPT);  

#Real_SaveAs_Cycles = 1 # Set this to use it in real experiment so that can really get capacity throughput!
V_max = 4.2;        V_min = 2.5;
cs_Neg_Init = 28920; Diff_SEI = 1.700000000000002e-20;    R_SEI = 2E5;   Bulk_Sol_Con = 4541.0;
Experiment_Long   = pybamm.Experiment( [ (f"Discharge at 1 C until {V_min} V", f"Charge at 0.3 C until {V_max} V", f"Hold at {V_max} V until C/5")    ] * 1 )  
Experiment_RPT    = pybamm.Experiment( [ (f"Discharge at 0.1 C until {V_min} V",  "Rest for 6 hours",  f"Charge at 0.1 C until {V_max} V" ) ] * 1 ) 
Experiment_Breakin= pybamm.Experiment( [ (f"Discharge at 0.2 C until {V_min} V","Rest for 4 hours",f"Charge at 0.2 C until {V_max} V" , f"Hold at {V_max} V until C/100","Rest for 2 hours") *5 + ( f"Discharge at 0.1 C until {V_min} V",  "Rest for 6 hours",  f"Charge at 0.1 C until {V_max} V" ) *1 ] )
#BasicPath = 'D:/OneDrive - Imperial College London/PhD Study/Imperial Study/Model/Pybamm Learning/SimDataSave/SolConsumed/';
#Target = 'Test 20-6 Update=78/';
#if not os.path.exists(BasicPath + Target):
#    os.mkdir(BasicPath + Target);


e:\allwork\python\pybamm\gitcode_pybamm\pybamm-gem\pybamm\experiments\experiment.py:82: DeprecationWarning: 'parameters' as an input to the Experiment class will soon be deprecated. Please open an issue if you are using this feature.
  warnings.warn(
e:\allwork\python\pybamm\gitcode_pybamm\pybamm-gem\pybamm\experiments\experiment.py:82: DeprecationWarning: 'parameters' as an input to the Experiment class will soon be deprecated. Please open an issue if you are using this feature.
  warnings.warn(
e:\allwork\python\pybamm\gitcode_pybamm\pybamm-gem\pybamm\experiments\experiment.py:82: DeprecationWarning: 'parameters' as an input to the Experiment class will soon be deprecated. Please open an issue if you are using this feature.
  warnings.warn(


In [4]:
# Run model without electrolyte dry out
ChemistryChen=pybamm.parameter_sets.Chen2020  # Ren2018  Chen2020_plating
Para_0=pybamm.ParameterValues(chemistry=ChemistryChen);
Para_0.update({'SEI resistivity [Ohm.m]':R_SEI}) ;
Para_0.update({'Outer SEI solvent diffusivity [m2.s-1]':Diff_SEI});  
Para_0.update({'Bulk solvent concentration [mol.m-3]':Bulk_Sol_Con});
Para_0.update({'Initial concentration in negative electrode [mol.m-3]':cs_Neg_Init }) # 
Sol_NonDry=[];  Model_NonDry=[];
# First Cycle 
Model_0  = pybamm.lithium_ion.DFN() # options={"SEI": "interstitial-diffusion limited","SEI film resistance": "distributed","SEI porosity change": "true" ,"lithium plating": "irreversible","lithium plating porosity change": "true"}
Sim_0    = pybamm.Simulation(Model_0, experiment = Experiment_Long,parameter_values = Para_0,solver = pybamm.CasadiSolver(),)       
Sol_0    = Sim_0.solve();

psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../c

SolverError: Maximum number of decreased steps occurred at t=0.0. Try solving the model up to this time only or reducing dt_max (currently, dt_max=3.5458164923888056).

In [ ]:
Model_0.options

In [ ]:
quick_plot = pybamm.QuickPlot(Sol_0) #     
quick_plot.dynamic_plot();


In [ ]:
"""Check that the model is not under- or over-determined."""
# Equations (differential and algebraic)
# Get all the variables from differential and algebraic equations
vars_in_rhs_keys = set()
vars_in_algebraic_keys = set()
vars_in_eqns = set()
# Get all variables ids from rhs and algebraic keys and equations, and
# from boundary conditions
# For equations we look through the whole expression tree.
# "Variables" can be Concatenations so we also have to look in the whole
# expression tree
unpacker = pybamm.SymbolUnpacker((pybamm.Variable, pybamm.VariableDot))

for var, eqn in Model_0.rhs.items():
    # Find all variables and variabledot objects
    vars_in_rhs_keys_dict = unpacker.unpack_symbol(var)
    vars_in_eqns_dict = unpacker.unpack_symbol(eqn)

    # Store ids only
    # Look only for Variable (not VariableDot) in rhs keys
    vars_in_rhs_keys.update(
        [
            var_id
            for var_id, var in vars_in_rhs_keys_dict.items()
            if isinstance(var, pybamm.Variable)
        ]
    )
    vars_in_eqns.update(vars_in_eqns_dict.keys())
for var, eqn in Model_0.algebraic.items():
    # Find all variables and variabledot objects
    vars_in_algebraic_keys_dict = unpacker.unpack_symbol(var)
    vars_in_eqns_dict = unpacker.unpack_symbol(eqn)

    # Store ids only
    # Look only for Variable (not VariableDot) in algebraic keys
    vars_in_algebraic_keys.update(
        [
            var_id
            for var_id, var in vars_in_algebraic_keys_dict.items()
            if isinstance(var, pybamm.Variable)
        ]
    )
    vars_in_eqns.update(vars_in_eqns_dict.keys())
for var, side_eqn in Model_0.boundary_conditions.items():
    for side, (eqn, typ) in side_eqn.items():
        vars_in_eqns_dict = unpacker.unpack_symbol(eqn)
        vars_in_eqns.update(vars_in_eqns_dict.keys())

# If any keys are repeated between rhs and algebraic then the model is
# overdetermined
if not set(vars_in_rhs_keys).isdisjoint(vars_in_algebraic_keys):
    print("model is overdetermined (repeated keys)")
# If any algebraic keys don't appear in the eqns (or bcs) then the model is
# overdetermined (but rhs keys can be absent from the eqns, e.g. dcdt = -1 is
# fine)
# Skip this step after discretisation, as any variables in the equations will
# have been discretised to slices but keys will still be variables
extra_algebraic_keys = vars_in_algebraic_keys.difference(vars_in_eqns)
# If any variables in the equations don't appear in the keys then the model is
# underdetermined
vars_in_keys = vars_in_rhs_keys.union(vars_in_algebraic_keys)
extra_variables_in_equations = vars_in_eqns.difference(vars_in_keys)

# get ids of external variables
external_ids = {var.id for var in Model_0.external_variables}
for var in Model_0.external_variables:
    if isinstance(var, pybamm.Concatenation):
        child_ids = {child.id for child in var.children}
        external_ids = external_ids.union(child_ids)

extra_variables = extra_variables_in_equations.difference(external_ids)

if extra_variables:
    print("model is underdetermined (too many variables)")


In [ ]:
for var, eqn in Model_0.rhs.items():
    # Find all variables and variabledot objects
    vars_in_rhs_keys_dict = unpacker.unpack_symbol(var)
    vars_in_eqns_dict = unpacker.unpack_symbol(eqn)
    print(vars_in_eqns_dict)
    print(vars_in_rhs_keys_dict)
    print('z')

In [ ]:
for var in Model_0.external_variables:
    print(var)

In [ ]:
print(Model_0.variable_names)
for key in Model_0._variables.keys():
    print(key)

In [ ]:
#print(Model_0.rhs.items())
for var, eqn in Model_0.rhs.items():
    print(var)
    #print(eqn)


In [ ]:
for var, eqn in Model_0.algebraic.items():
    print(var)

In [ ]:
print(extra_variables_in_equations)
print(extra_variables)
print(external_ids)

In [ ]:
Model_0.submodels